In [ ]:
import pandas as pd
import json
import urllib.request as urlreq
import os

In [ ]:
def json_to_qry(url_to_json):
    json_url = urlreq.urlopen(url_to_json)
    json_file = json.loads(json_url.read())
    qry_lst = []
    def add_quote(lst):
        lst_quote = ["'"+str(x)+"'" for x in lst]
        return (lst_quote)
    for k,v in json_file.items():
        for cd,sig in v.items():
            if cd=='long': continue
            # entail the range
            if 'range' in sig:
                for x in sig['range']:
                    key_quote = [str(y) for y in list(range(int(x.split('-')[0]),int(x.split('-')[1])+1))]
                    sig['exact'].extend(key_quote)

            # generate dynamic queries
            qry = '''
                select ''' + "'" + k + "'" + ''' as CD_GRP, 
                       ''' + "'" + v['long'] + "'" + ''' as CD_GRP_LONG,
                       concept_id,concept_name,concept_code,vocabulary_id,domain_id
                from concept
                where vocabulary_id = '''+ "'" + cd.upper() + "'" +''' and
            '''
            if 'icd' in cd and 'pcs' not in cd:
                where_lev0 = '''substring_index(concept_code,'.',1) in ('''+ ','.join(add_quote(sig['lev0'])) +''')''' if sig['lev0'] else None
                where_lev1 = '''substring(concept_code,1,5) in ('''+ ','.join(add_quote(sig['lev1'])) +''')''' if sig['lev1'] else None
                where_lev2 = '''substring(concept_code,1,6) in ('''+ ','.join(add_quote(sig['lev2'])) +''')''' if sig['lev2'] else None
                where_nonempty = [s for s in [where_lev0,where_lev1,where_lev2] if s is not None]

                qry += '''
                (
                     ''' + ' or '.join(where_nonempty) + '''  
                )         
                '''
            else:
                qry += '''
                (
                     concept_code in ('''+ ','.join(add_quote(sig['exact'])) +''')
                )         
                '''
            qry_lst.append(qry)
            
    return qry_lst

In [ ]:
# load delivery codes
qry2_lst = json_to_qry('https://raw.githubusercontent.com/RWD2E/phecdm/main/res/valueset_curated/vs-mmm-comorb.json')

qry2_final = ' union all '.join(qry2_lst)
print(qry2_final)
comorb_omop_cd = spark.sql(' union all '.join(qry2_lst)).toPandas()
comorb_omop_cd.to_csv(os.path.join(path_to_data,'cd_meta_omop_comorb.csv'),index=False)

# comorb_omop_meta = spark.sql(' union all '.join(qry2_lst)).cache()
# comorb_omop_meta.createOrReplaceTempView("comorb_omop_meta")
# comorb_omop_meta.first()

In [ ]:
# load SMM code list and get omop concept_id
qry_lst = json_to_qry('https://raw.githubusercontent.com/RWD2E/phecdm/main/res/valueset_curated/vs-mmm.json')

qry_final = ' union all '.join(qry_lst)
print(qry_final)
# smm_omop_cd = spark.sql(' union all '.join(qry_lst)).toPandas()
# smm_omop_cd.to_csv(os.path.join(path_to_data,'cd_meta_omop_smm.csv'),index=False)

# smm_omop_meta = spark.sql(' union all '.join(qry_lst)).cache()
# smm_omop_meta.createOrReplaceTempView("smm_omop_meta")
# smm_omop_meta.first()

In [ ]:
# load comorb code list and get omop concept_id
qry2_lst = json_to_qry('https://raw.githubusercontent.com/RWD2E/phecdm/main/res/valueset_curated/vs-mmm-comorb.json')

qry2_final = ' union all '.join(qry2_lst)
print(qry2_final)
comorb_omop_cd = spark.sql(' union all '.join(qry2_lst)).toPandas()
comorb_omop_cd.to_csv(os.path.join(path_to_data,'cd_meta_omop_comorb.csv'),index=False)

# comorb_omop_meta = spark.sql(' union all '.join(qry2_lst)).cache()
# comorb_omop_meta.createOrReplaceTempView("comorb_omop_meta")
# comorb_omop_meta.first()